In [1]:
import pandas as pd
import numpy as np
!pip install folium
import folium as fl
from geopy.geocoders import Nominatim
import requests
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors


    100% |████████████████████████████████| 92kB 6.2MB/s eta 0:00:01
Requirement not upgraded as not directly required: jinja2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: numpy in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: MarkupSafe>=0.23 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from jinja2->folium)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->folium)
Requirement not upgraded as not directly required: idna<2.7,>=2.5 in /opt/conda/envs/DSX-Python35/

### Dataframe from previous exercise

In [2]:
tables = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
df = tables[0][1:]
df.columns = tables[0].iloc[0]

In [3]:
df.is_copy = False
df.loc(df['Borough'].replace('Not assigned', np.nan, inplace=True))
df.is_copy = False
df.dropna(subset=['Borough'], inplace=True)

In [4]:
df['Neighbourhood'].mask(df['Neighbourhood'] == 'Not assigned', df['Borough'], inplace=True)

In [5]:
df_clean = df.groupby(["Postcode","Borough"])['Neighbourhood'].apply(lambda tags: ','.join(tags)).reset_index(name = "Neighbourhood")

In [6]:
print('Shape After data cleanup:',df_clean.shape)

Shape After data cleanup: (103, 3)


In [7]:
df_clean.query('Postcode == "M5A"')

,Postcode,Borough,Neighbourhood
53,M5A,Downtown Toronto,"Harbourfront,Regent Park"


### Filter Toronto Borough 

In [8]:
neighborhoods_toronto = df[df['Borough'].str.contains(r'Toronto')]
print("Toronto neighborhoods:", neighborhoods_toronto.shape)
neighborhoods_toronto.head()

Toronto neighborhoods: (74, 3)


,Postcode,Borough,Neighbourhood
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
18,M5B,Downtown Toronto,Ryerson
19,M5B,Downtown Toronto,Garden District
35,M5C,Downtown Toronto,St. James Town


### Merge geo data 

In [19]:
geolocator = Nominatim(user_agent="ny_explorer")
neighborhoods = pd.DataFrame(columns = ['Borough', 'Neighbourhood', 'Latitude', 'Longitude'])
for hood, borough in zip(neighborhoods_toronto['Neighbourhood'],neighborhoods_toronto['Borough']):
    #print(hood+', Toronto')
    location = geolocator.geocode(hood+', Toronto')  
    if (location is not None): neighborhoods.loc[len(neighborhoods)] = [borough, hood, location.latitude, location.longitude]
print(neighborhoods.shape)
neighborhoods.head()

(72, 4)


,Borough,Neighbourhood,Latitude,Longitude
0,Downtown Toronto,Harbourfront,43.640080,-79.380150
1,Downtown Toronto,Regent Park,43.660706,-79.360457
2,Downtown Toronto,Ryerson,43.621573,-79.559130
3,Downtown Toronto,Garden District,43.656502,-79.377128
4,Downtown Toronto,St. James Town,43.669403,-79.372704


## Maps data 

In [20]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [21]:
# create map of Toronto using latitude and longitude values
map_toronto = fl.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = fl.Popup(label, parse_html=True)
    fl.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

##  Foursquare venues data collection 

In [22]:
# @hidden_cell
CLIENT_ID = 'LCLH3TP21APF0YU5LZE2O3ZRP3GWXCYL5Z5P1UECNVBQJVJZ' # your Foursquare ID
CLIENT_SECRET = 'KDWYYMPX3NQ5GNXWM1EQ3143XYSOQH5K45EDPPVQLCVTDDRH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Foursquare Credentails')
print('CLIENT_ID: ' + CLIENT_ID)

Foursquare Credentails
CLIENT_ID: LCLH3TP21APF0YU5LZE2O3ZRP3GWXCYL5Z5P1UECNVBQJVJZ


In [24]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Explore Neighborhoods in Toronto

In [25]:
toronto_venues = getNearbyVenues(names=neighborhoods['Neighbourhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

#### Let's check the size of the resulting venues dataframe

In [26]:
print(toronto_venues.shape)
toronto_venues.head()

(3635, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.64008,-79.38015,Harbour Square Park,43.639253,-79.378395,Park
1,Harbourfront,43.64008,-79.38015,Lake Ontario,43.638945,-79.379665,Lake
2,Harbourfront,43.64008,-79.38015,Harbourfront,43.639526,-79.380688,Neighborhood
3,Harbourfront,43.64008,-79.38015,Miku,43.641374,-79.377531,Japanese Restaurant
4,Harbourfront,43.64008,-79.38015,BeaverTails,43.639899,-79.380197,Bakery


In [27]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 300 uniques categories.


### Analyze Each Neighborhood

In [28]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
print(toronto_onehot.shape)
toronto_onehot.head()

(3635, 301)


,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo
0,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [29]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
print(toronto_grouped.shape)
toronto_grouped.head()

(71, 301)


,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo
0,Adelaide,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.01,0.0,0.0,0.00,0.0,0.010000,0.0,0.0,0.000000,0.0
1,Bathurst Quay,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.043478,...,0.00,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.000000,0.0
2,Berczy Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.01,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.000000,0.0
3,Brockton,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.00,0.0,0.0,0.15,0.0,0.000000,0.0,0.0,0.000000,0.0
4,CN Tower,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.00,0.0,0.0,0.00,0.0,0.011765,0.0,0.0,0.011765,0.0


#### Let's print each neighborhood along with the top 5 most common venues

In [30]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
            venue  freq
0     Coffee Shop  0.09
1           Hotel  0.05
2       Gastropub  0.04
3  Cosmetics Shop  0.04
4            Café  0.04


----Bathurst Quay----
           venue  freq
0    Coffee Shop  0.17
1           Café  0.13
2           Park  0.09
3  Grocery Store  0.04
4         Tunnel  0.04


----Berczy Park----
         venue  freq
0  Coffee Shop  0.09
1         Café  0.06
2   Restaurant  0.05
3       Bakery  0.04
4        Hotel  0.04


----Brockton----
                   venue  freq
0                    Bar  0.15
1  Vietnamese Restaurant  0.15
2                   Park  0.10
3            Coffee Shop  0.05
4      French Restaurant  0.05


----CN Tower----
                venue  freq
0               Hotel  0.09
1         Coffee Shop  0.09
2         Pizza Place  0.06
3            Aquarium  0.05
4  Italian Restaurant  0.05


----Cabbagetown----
         venue  freq
0  Coffee Shop  0.08
1   Restaurant  0.08
2         Café  0.06
3       Bakery  0.04
4  Pizza P

          venue  freq
0     Gift Shop  0.05
1  Gourmet Shop  0.05
2     Bookstore  0.05
3    Restaurant  0.05
4          Café  0.05


----Rosedale----
          venue  freq
0          Park  0.50
1    Bike Trail  0.25
2    Playground  0.25
3     Nightclub  0.00
4  Optical Shop  0.00


----Roselawn----
              venue  freq
0  Asian Restaurant   0.2
1              Café   0.2
2      Skating Rink   0.2
3            Garden   0.2
4          Pharmacy   0.2


----Runnymede----
               venue  freq
0        Pizza Place  0.12
1        Coffee Shop  0.08
2  Convenience Store  0.04
3             Bakery  0.04
4               Bank  0.04


----Ryerson----
                    venue  freq
0     Sporting Goods Shop  0.07
1               Pet Store  0.07
2            Burger Joint  0.07
3  Furniture / Home Store  0.07
4               Gift Shop  0.07


----South Hill----
            venue  freq
0  Sandwich Place  0.10
1  History Museum  0.10
2     Coffee Shop  0.10
3            Café  0.07
4     Piz

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [69]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Hotel,Cosmetics Shop,Café,Gastropub,Restaurant,American Restaurant,Salad Place,Japanese Restaurant,Breakfast Spot
1,Bathurst Quay,Coffee Shop,Café,Park,Grocery Store,Diner,Caribbean Restaurant,Garden,Sculpture Garden,Dance Studio,Ramen Restaurant
2,Berczy Park,Coffee Shop,Café,Restaurant,Hotel,Bakery,Gastropub,Seafood Restaurant,Beer Bar,Italian Restaurant,Japanese Restaurant
3,Brockton,Bar,Vietnamese Restaurant,Park,Grocery Store,Dive Bar,Korean Restaurant,South American Restaurant,Gastropub,Organic Grocery,French Restaurant
4,CN Tower,Coffee Shop,Hotel,Pizza Place,Italian Restaurant,Aquarium,Scenic Lookout,Gym,Concert Hall,Baseball Stadium,Restaurant


## Cluster Toronto Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [67]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 4, 4, 0, 4, 4, 3, 0, 0, 4], dtype=int32)

In [70]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged =toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,Harbourfront,43.640080,-79.380150,4,Coffee Shop,Café,Hotel,Italian Restaurant,Restaurant,Pizza Place,Bakery,Gym,Sporting Goods Shop,Sports Bar
1,Downtown Toronto,Regent Park,43.660706,-79.360457,4,Coffee Shop,Thai Restaurant,Pub,Electronics Store,Fast Food Restaurant,Beer Store,Food Truck,Sushi Restaurant,Auto Dealership,Restaurant
2,Downtown Toronto,Ryerson,43.621573,-79.559130,4,Department Store,Coffee Shop,Arts & Crafts Store,Restaurant,Gift Shop,Burger Joint,Furniture / Home Store,Gym,Discount Store,Portuguese Restaurant
3,Downtown Toronto,Garden District,43.656502,-79.377128,4,Coffee Shop,Clothing Store,Restaurant,Cosmetics Shop,Café,Fast Food Restaurant,Middle Eastern Restaurant,Hotel,Sporting Goods Shop,Japanese Restaurant
4,Downtown Toronto,St. James Town,43.669403,-79.372704,4,Coffee Shop,Pizza Place,Grocery Store,Café,Diner,Library,Bar,Bank,Bakery,Restaurant


Finally, let's visualize the resulting clusters

In [71]:
# create map
map_clusters = fl.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = fl.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    fl.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters